In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install timm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer

<ipython-input-2-2540d8c9d86b>:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
class CFG:
    debug = False
    # image_path = image_path
    # captions_path = captions_path
    batch_size = 32
    num_workers = 2
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_name = 'vit_base_patch32_clip_224.openai'
    image_embedding = 768
    text_encoder_model = "distilbert-base-uncased"
    text_embedding = 768
    text_tokenizer = "distilbert-base-uncased"
    max_length = 200

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 224

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

In [4]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = captions
        self.encoded_captions = tokenizer(
            captions, padding=True, truncation=True, max_length=CFG.max_length
        )

        self.transforms = transforms

    def __getitem__(self, idx):

        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }


        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']

        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['productDisplayName'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

In [5]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [6]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [7]:
class ProjectionHead_img(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        print("Image emb:",embedding_dim)
        print("Proj dem",projection_dim)
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, x):
        print(x.shape)
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

class ProjectionHead_txt(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        print("Text emb:",embedding_dim)
        print("Proj dem",projection_dim)
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, x):
        print("Input Feature:",x.shape)
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [8]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead_img(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead_txt(embedding_dim=text_embedding)
        self.temperature = temperature


    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )

        # Getting Image and Text Embeddings (with same dimension)
        print("Features:",image_features.shape,text_features.shape)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        print("Embeddings:",image_embeddings.shape,text_embeddings.shape)
        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / (2 * self.temperature), dim=-1
        )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [9]:
def make_train_valid_dfs():
    dataframe = pd.read_csv(f"{CFG.captions_path}/captions.csv")
    # print(dataframe.head())
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.2 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe


def build_loaders(dataframe, tokenizer, mode):

    transforms = get_transforms(mode=mode)
    # print(dataframe["productDisplayName"].tolist())
    image_paths = dataframe["image"].astype('str').tolist()
    texts = dataframe["productDisplayName"].astype('str').tolist()
    dataset = CLIPDataset(
        image_paths,
        texts,
        tokenizer=tokenizer,
        transforms=transforms,
    )

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        shuffle=True if mode == "train" else False,
    )
    return dataloader

In [10]:
def get_image_embeddings(valid_df, model_path):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")
    
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()
    
    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

In [12]:
# _, valid_df = make_train_valid_dfs()
# model, image_embeddings = get_image_embeddings(valid_df, "/content/drive/MyDrive/Saved_Model/CLIP_model_VIT_Fashion.pt")


    



In [13]:
def find_matches(model, image_embeddings, query, image_filenames, n=9):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    encoded_query = tokenizer([query])
    batch = {
        key: torch.tensor(values).to(CFG.device)
        for key, values in encoded_query.items()
    }
    with torch.no_grad():
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        text_embeddings = model.text_projection(text_features)
    
    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    dot_similarity = text_embeddings_n @ image_embeddings_n.T
    
    values, indices = torch.topk(dot_similarity.squeeze(0), n * 5)
    matches = [image_filenames[idx] for idx in indices[::5]]
    
    _, axes = plt.subplots(3, 3, figsize=(10, 10))
    for match, ax in zip(matches, axes.flatten()):
        image = cv2.imread(f"{CFG.image_path}/{match}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis("off")
    
    plt.show()

In [11]:
!pip install -qU datasets transformers sentence-transformers \
                 pinecone-client pinecone-text protobuf==3.20.3

In [12]:
import os
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("3c1215ea-8adb-4c61-a39a-e605bf37be57") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
env = os.getenv("asia-southeast1-gcp-free") or "PINECONE_ENVIRONMENT"

# init connection to pinecone
pinecone.init(
    api_key=api_key,
    environment=env
)

In [13]:
# # choose a name for your index
# index_name = "hybrid"

# if index_name not in pinecone.list_indexes():
#     # create the index
#     pinecone.create_index(
#       index_name,
#       dimension=512,
#       metric="dotproduct",
#       pod_type="Starter"
#     )
# pineconeIndex = pinecone.GRPCIndex(index_name)


## Connect with the pine cone
import pinecone
# init connection to pinecone
pinecone.init(
    api_key="3c1215ea-8adb-4c61-a39a-e605bf37be57", # app.pinecone.io
    environment="asia-southeast1-gcp-free" # find next to api key
)

index_name = "hybrid"

if index_name not in pinecone.list_indexes():
# create the index
 pinecone.create_index(
    index_name,
    dimension=512,
    metric="dotproduct",
    pod_type="Starter")

# pineconeIndex = pinecone.GRPCIndex(index_name)


In [14]:
index = pinecone.Index(index_name)

In [15]:
from datasets import load_dataset

# load the dataset from huggingface datasets hub
fashion = load_dataset(
    "ashraq/fashion-product-images-small",
    split="train"
)
fashion

Dataset({
    features: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName', 'image'],
    num_rows: 44072
})

In [25]:
# assign the images and metadata to separate variables
images_Product = fashion["productDisplayName", "image"]
# metadata = fashion.remove_columns("image")

ValueError: ignored

In [20]:
# display a product image
images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FBFD6BF0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E6AA0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=L size=60x80 at 0x7F27FB4E6B00>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E6E30>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E6E90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E6F20>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E7160>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E71F0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E7280>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E74C0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E77C0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F27FB4E7A00>,
 <PIL.JpegImagePlu

In [21]:
# convert metadata into a pandas dataframe
metadata = metadata.to_pandas()
metadata.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [22]:
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
"Turtle Check Men Navy Blue Shirt".lower().split()

['turtle', 'check', 'men', 'navy', 'blue', 'shirt']

In [24]:
bm25.fit(metadata['productDisplayName'])

  0%|          | 0/44072 [00:00<?, ?it/s]

In [25]:
metadata['productDisplayName'][0]

'Turtle Check Men Navy Blue Shirt'

In [26]:
bm25.encode_queries(metadata['productDisplayName'][0])

{'indices': [23789636,
  1830646559,
  632192512,
  931643408,
  3905155331,
  3828986392],
 'values': [0.3276687848622804,
  0.19377339510596148,
  0.040475545164610806,
  0.1808640794607714,
  0.10791423980552016,
  0.1493039556008558]}

In [27]:
bm25.encode_documents(metadata['productDisplayName'][0])

{'indices': [23789636,
  1830646559,
  632192512,
  931643408,
  3905155331,
  3828986392],
 'values': [0.4449638258432887,
  0.4449638258432887,
  0.4449638258432887,
  0.4449638258432887,
  0.4449638258432887,
  0.4449638258432887]}

In [28]:
dense_vec = model.encode([metadata['productDisplayName'][0]])
dense_vec.shape

NameError: ignored

In [29]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load a CLIP model from huggingface
model = SentenceTransformer(
    'sentence-transformers/clip-ViT-B-32',
    device=device
)
model


SentenceTransformer(
  (0): CLIPModel()
)

In [16]:
model_path="/content/drive/MyDrive/Saved_Model/CLIP_model_VIT_Fashion.pt"
model_finetuned = CLIPModel().to(CFG.device)
model_finetuned.load_state_dict(torch.load(model_path, map_location=CFG.device))
model_finetuned.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Image emb: 768
Proj dem 256
Text emb: 768
Proj dem 256


CLIPModel(
  (image_encoder): ImageEncoder(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [31]:
meta_batch = metadata.iloc[0:10]
meta_dict = meta_batch.to_dict(orient="records")
# concatinate all metadata field except for id and year to form a single string
meta_batch = [" ".join(x) for x in meta_batch.loc[:, ~meta_batch.columns.isin(['id', 'year'])].values.tolist()]
# extract image batch
img_batch = images[0:10]
# create sparse BM25 vectors
sparse_embeds = bm25.encode_documents([text for text in meta_batch])
# create dense vectors
dense_embeds = model.encode(img_batch).tolist()

In [35]:
import pandas as pd
h=fashion[:20]
q=pd.DataFrame(h)

# g=fashion.to_pandas()
# f=h['image']
q=h['image']
q

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE6B0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE0E0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=L size=60x80 at 0x7FCA7E0BC940>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BD540>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BDED0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE2F0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE8C0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BDCC0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE4D0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BCE50>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BDC90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7FCA7E0BE4A0>,
 <PIL.JpegImagePlu

In [ ]:
fashion.iloc[0:10]

In [ ]:
len(dense_embeds[0])

In [32]:



class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = captions
        self.encoded_captions = tokenizer(
            captions, padding=True, truncation=True, max_length=CFG.max_length
        )

        self.transforms = transforms

    def __getitem__(self, idx):

        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }


        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']

        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['productDisplayName'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
        
        
        


In [33]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead_img(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead_txt(embedding_dim=text_embedding)
        self.temperature = temperature


    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )

        # Getting Image and Text Embeddings (with same dimension)
        print("Features:",image_features.shape,text_features.shape)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        print("Embeddings:",image_embeddings.shape,text_embeddings.shape)
        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / (2 * self.temperature), dim=-1
        )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [56]:
import pandas as pd


class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = captions
        self.encoded_captions = tokenizer(
            captions, padding=True, truncation=True, max_length=CFG.max_length
        )

        self.transforms = transforms

    def __getitem__(self, idx):

        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }


        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']

        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['productDisplayName'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)



        
        

from PIL import Image 
import torchvision.transforms as transforms
class CLIP_Img_Dataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames,captions, tokenizer):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """
        self.image_filenames = image_filenames
        self.captions = captions
        self.encoded_captions = tokenizer(
            captions, padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        

    def __getitem__(self, idx):

        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        # Convert the bytes to a string
        # image_data = self.image_filenames.decode('latin-1')

        # Create a numpy array from the string data
        # print(self.image_filenames[idx])
        # image_array = np.frombuffer(self.image_filenames[idx])
        # image = cv2.cvtColor(np.array(self.image_filenames[idx]), cv2.COLOR_RGB2BGR)
        # Decode the image array using cv2.imdecode
        # image = cv2.imdecode(self.image_filenames[idx], cv2.IMREAD_COLOR)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # np_image = np.array(self.image_filenames[idx])

        # Convert NumPy array to OpenCV format
        # open_cv_image = cv2.cvtColor(np_image.astype(bytes), cv2.COLOR_RGB2BGR)
        image=self.transform(self.image_filenames[idx].convert("RGB"))

        # item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['image'] = torch.tensor(image)
        item['productDisplayName'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)


# def build_img_loaders(dataframe, tokenizer, mode):

#     transforms = get_transforms(mode=mode)
#     # print(dataframe["productDisplayName"].tolist())
#     image_paths = dataframe["image"].tolist()
#     texts = dataframe["productDisplayName"].astype('str').tolist()
#     dataset = CLIP_Img_Dataset(
#         image_paths,
#         texts,
#         tokenizer=tokenizer,
#         transforms=transforms,
#     )

#     dataloader = torch.utils.data.DataLoader(
#         dataset,
#         batch_size=CFG.batch_size,
#         num_workers=0,
#     )
#     return dataloader



def build_img_loaders(fashion_batch, tokenizer):

    # transforms = get_transforms(mode=mode)
    # print(dataframe["productDisplayName"].tolist())
    image_paths = fashion_batch["image"].tolist()
    texts = [" ".join(x) for x in fashion_batch.loc[:, ~fashion_batch.columns.isin(['id', 'year','image'])].values.tolist()]
    dataset = CLIP_Img_Dataset(
        image_paths,
        texts,
        tokenizer=tokenizer
    )

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=0
    )
    return dataloader


def encode_img_txt(fashion_batch):
    image_df=fashion_batch['image']
    fashion_batch=pd.DataFrame(fashion_batch)
    txt_batch = [" ".join(x) for x in fashion_batch.loc[:, ~fashion_batch.columns.isin(['id', 'year','image'])].values.tolist()]
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    text_embeddings_list=[]
    for query in txt_batch:
        encoded_query = tokenizer([query])
        batch = {
            key: torch.tensor(values).to(CFG.device)
            for key, values in encoded_query.items()
        }
        with torch.no_grad():
            text_features = model_finetuned.text_encoder(
                input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
            )
        text_embeddings = model_finetuned.text_projection(text_features)
        text_embeddings_list.append(text_embeddings)



    valid_loader = build_img_loaders(fashion_batch, tokenizer)
    image_embeddings_list=[]
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model_finetuned.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model_finetuned.image_projection(image_features)
            image_embeddings_list.append(image_embeddings)
    return image_embeddings_list,text_embeddings_list

from tqdm.auto import tqdm

batch_size = 200



for i in tqdm(range(0, len(fashion), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(fashion))
    # extract metadata batch
    fashion_batch = fashion[i:i_end]
    # meta_dict = fashion_batch.to_dict(orient="records")
    # concatinate all metadata field except for id and year to form a single string
    # fashion_batch = [" ".join(x) for x in fashion_batch.loc[:, ~fashion_batch.columns.isin(['id', 'year'])].values.tolist()]
    # img_batch = image_df[i:i_end]
    dense_embeds,sparse_embeds=encode_img_txt(fashion_batch)
    print(dense_embeds,sparse_embeds)

    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]

    upserts = []
    # loop through the data and create dictionaries for uploading documents to pinecone index
    for _id, sparse, dense, meta in zip(ids, sparse_embeds, dense_embeds, meta_dict):
        upserts.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': meta
        })
    # upload the documents to the new hybrid index
    index.upsert(upserts)

# show index description after uploading the documents
index.describe_index_stats()

# print(encode_img_txt(fashion_batch))

# def get_image_embeddings(valid_df, model_path):
#     tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
#     valid_loader = build_loaders(valid_df, tokenizer, mode="valid")
    
#     model = CLIPModel().to(CFG.device)
#     model.load_state_dict(torch.load(model_path, map_location=CFG.device))
#     model.eval()
    
#     valid_image_embeddings = []
#     with torch.no_grad():
#         for batch in tqdm(valid_loader):
#             image_features = model.image_encoder(batch["image"].to(CFG.device))
#             image_embeddings = model.image_projection(image_features)
#             valid_image_embeddings.append(image_embeddings)
#     return model, torch.cat(valid_image_embeddings)

  0%|          | 0/221 [00:00<?, ?it/s]

Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1, 768])
Input Feature: torch.Size([1

  0%|          | 0/7 [00:00<?, ?it/s]

<ipython-input-56-3abb3d1c9bc9>:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['image'] = torch.tensor(image)


Streaming output truncated to the last 5000 lines.
          1.3402,  0.2302, -0.2437,  0.3068, -1.2903, -2.7290, -0.0458, -0.0543,
         -2.6715, -0.6743, -0.9164,  0.0616, -0.9943, -0.4459, -1.0458, -0.9077,
         -0.7037,  0.1231,  0.6775, -0.6528, -0.1742,  1.2251, -0.7534, -0.8105,
          0.2291, -2.6288, -1.6097,  1.0534,  0.6604, -0.5389,  0.4546, -0.9534,
          0.1485, -0.3968,  1.9317, -0.2865,  1.9065,  0.1231,  0.0131, -0.1481,
          2.0598, -1.1709, -0.7016, -1.0445, -0.1604, -1.6182, -0.4863, -1.0666,
          1.5875,  1.4487, -2.2180, -0.4945,  2.1020, -1.0509, -1.8642, -1.5556,
         -0.4113,  0.5529,  0.3403, -1.2159, -0.3117,  2.0499, -0.9945, -1.5896,
          0.0788, -0.5200, -0.1943, -0.3829,  0.4794, -0.9785,  0.5155,  0.5623,
         -0.7727, -0.5721,  2.1712,  1.2260,  0.6091,  0.3656,  1.6341, -0.0092,
          0.7891,  0.7748, -0.1871,  0.6555, -0.8973,  0.7435,  0.4349,  0.3199,
         -0.4528, -0.7744, -0.1750, -1.7806,  0.1740,  0.8

ValueError: ignored

In [ ]:
from tqdm.auto import tqdm

batch_size = 200

for i in tqdm(range(0, len(fashion), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(fashion))
    # extract metadata batch
    meta_batch = metadata.iloc[i:i_end]
    meta_dict = meta_batch.to_dict(orient="records")
    # concatinate all metadata field except for id and year to form a single string
    meta_batch = [" ".join(x) for x in meta_batch.loc[:, ~meta_batch.columns.isin(['id', 'year'])].values.tolist()]
    # extract image batch
    img_batch = images[i:i_end]
    # create sparse BM25 vectors
    sparse_embeds = bm25.encode_documents([text for text in meta_batch])
    # create dense vectors
    dense_embeds = model.encode(img_batch).tolist()
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]

    upserts = []
    # loop through the data and create dictionaries for uploading documents to pinecone index
    for _id, sparse, dense, meta in zip(ids, sparse_embeds, dense_embeds, meta_dict):
        upserts.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': meta
        })
    # upload the documents to the new hybrid index
    index.upsert(upserts)

# show index description after uploading the documents
index.describe_index_stats()

In [ ]:
query = "dark blue french connection jeans for men"

# create sparse and dense vectors
sparse = bm25.encode_queries(query)
dense = model.encode(query).tolist()
# search
result = index.query(
    top_k=14,
    vector=dense,
    sparse_vector=sparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
imgs

In [ ]:
from IPython.core.display import HTML
from io import BytesIO
from base64 import b64encode

# function to display product images
def display_result(image_batch):
    figures = []
    for img in image_batch:
        b = BytesIO()  
        img.save(b, format='png')
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="data:image/png;base64,{b64encode(b.getvalue()).decode('utf-8')}" style="width: 90px; height: 120px" >
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [ ]:
display_result(imgs)

In [ ]:
def hybrid_scale(dense, sparse, alpha: float):
    """Hybrid vector scaling using a convex combination

    alpha * dense + (1 - alpha) * sparse

    Args:
        dense: Array of floats representing
        sparse: a dict of `indices` and `values`
        alpha: float between 0 and 1 where 0 == sparse only
               and 1 == dense only
    """
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    # scale sparse and dense vectors to create hybrid search vecs
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    return hdense, hsparse

In [ ]:
question = "dark blue french connection jeans for men"

# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
for x in result["matches"]:
    print(x["metadata"]['productDisplayName'])

In [ ]:
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=1)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
for x in result["matches"]:
    print(x["metadata"]['productDisplayName'])

In [ ]:
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.05)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
for x in result["matches"]:
    print(x["metadata"]['productDisplayName'])

In [ ]:
query = "small beige handbag for women"
# create sparse and dense vectors
sparse = bm25.encode_queries(query)
dense = model.encode(query).tolist()
# scale sparse and dense vectors - keyword search first
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.05)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
for x in result["matches"]:
    print(x["metadata"]['productDisplayName'])

In [ ]:
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=1)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
images[36254]

In [ ]:
query = "soft purple topwear"
# create the sparse vector
sparse = bm25.encode_queries(query)
# now create the dense vector using the image
dense = model.encode(images[36254]).tolist()
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.3)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True
)
# use returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
query = "soft purple topwear"
# create the sparse vector
sparse = bm25.encode_queries(query)
# now create the dense vector using the image
dense = model.encode(images[36254]).tolist()
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.3)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True,
    filter={"baseColour": "Purple"}  # add to metadata filter
)
# used returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
images[36256]

In [ ]:
query = "soft green color topwear"
# create the sparse vector
sparse = bm25.encode_queries(query)
# now create the dense vector using the image
dense = model.encode(images[36256]).tolist()
# scale sparse and dense vectors
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.6)
# search
result = index.query(
    top_k=14,
    vector=hdense,
    sparse_vector=hsparse,
    include_metadata=True,
    filter={"baseColour": "Green"}  # add to metadata filter
)
# use returned product ids to get images
imgs = [images[int(r["id"])] for r in result["matches"]]
# display the images
display_result(imgs)

In [ ]:
pinecone.delete_index(index_name)